# **Imports**

In [31]:
from sklearn import svm

In [34]:
from sklearn.linear_model import SGDClassifier

In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.1 MB/s 


In [2]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import lightgbm as lgb

import ast

# **Loading and Preprocessing**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip '/content/drive/MyDrive/Datasets/Network Intrusion Detection.zip' -d '/content/'

Archive:  /content/drive/MyDrive/Datasets/Network Intrusion Detection.zip
  inflating: /content/Test_data.csv  
  inflating: /content/Train_data.csv  


In [38]:
TRAIN_PATH = "/content/Train_data.csv"
TEST_PATH = "/content/Test_data.csv"

In [39]:
train_data = pd.read_csv(TRAIN_PATH)
train_data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25187,0,tcp,exec,RSTO,0,0,0,0,0,0,...,7,0.03,0.06,0.00,0.00,0.00,0.00,1.00,1.00,anomaly
25188,0,tcp,ftp_data,SF,334,0,0,0,0,0,...,39,1.00,0.00,1.00,0.18,0.00,0.00,0.00,0.00,anomaly
25189,0,tcp,private,REJ,0,0,0,0,0,0,...,13,0.05,0.07,0.00,0.00,0.00,0.00,1.00,1.00,anomaly
25190,0,tcp,nnsp,S0,0,0,0,0,0,0,...,20,0.08,0.06,0.00,0.00,1.00,1.00,0.00,0.00,anomaly


In [40]:
test_data = pd.read_csv(TEST_PATH)

In [41]:
test_data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,private,REJ,0,0,0,0,0,0,...,255,10,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00
1,0,tcp,private,REJ,0,0,0,0,0,0,...,255,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,134,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,3,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,29,86,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,smtp,SF,794,333,0,0,0,0,...,100,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00
22540,0,tcp,http,SF,317,938,0,0,0,0,...,197,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00
22541,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07
22542,0,udp,domain_u,SF,42,42,0,0,0,0,...,255,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00


In [42]:
result = pd.concat([train_data, test_data])

In [43]:
result = result.reset_index()
result = result.drop(columns=['index'])

In [44]:
result

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47731,0,tcp,smtp,SF,794,333,0,0,0,0,...,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,NaN
47732,0,tcp,http,SF,317,938,0,0,0,0,...,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,NaN
47733,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,NaN
47734,0,udp,domain_u,SF,42,42,0,0,0,0,...,252,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,NaN


In [45]:
# must be changed
LABEL_COLUMN = 'class'

In [46]:
result.iloc[100]

duration                            0
protocol_type                     tcp
service                          http
flag                               SF
src_bytes                           0
dst_bytes                         354
land                                0
wrong_fragment                      0
urgent                              0
hot                                 0
num_failed_logins                   0
logged_in                           0
num_compromised                     0
root_shell                          0
su_attempted                        0
num_root                            0
num_file_creations                  0
num_shells                          0
num_access_files                    0
num_outbound_cmds                   0
is_host_login                       0
is_guest_login                      0
count                              32
srv_count                          32
serror_rate                      0.03
srv_serror_rate                  0.03
rerror_rate 

In [47]:
result['protocol_type'] = pd.factorize(result['protocol_type'], sort=True)[0]

In [48]:
result['service'] = pd.factorize(result['service'], sort=True)[0]

In [49]:
result['class'] = pd.factorize(result['class'], sort=True)[0]

In [50]:
result['flag'] = pd.factorize(result['flag'], sort=True)[0]

In [51]:
def standardize(df, column):
  df[column] = (df[column] - df[column].mean()) / df[column].std()

In [52]:
standardize(result, 'service')
standardize(result, 'dst_bytes')
standardize(result, 'count')
standardize(result, 'srv_count')
standardize(result, 'dst_host_count')
standardize(result, 'dst_host_srv_count')
standardize(result, 'src_bytes')

In [53]:
train_data = result[:len(train_data)]
test_data = result[len(train_data):]

In [54]:
cols = ['duration',
 'protocol_type',
 'service',
 'flag',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate']

In [55]:
print(cols)

['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']


In [56]:
X = train_data[cols]
y = train_data['class']

In [57]:
test_data = test_data[cols]

# **Models!**

In [58]:
from keras.models import Sequential
from keras.layers import Activation, Dense

In [59]:
model = Sequential()
model.add(Dense(12, input_shape=(len(cols),), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X, y, epochs=10, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/10
2520/2520 [==============================] - 6s 2ms/step - loss: 0.3756 - accuracy: 0.9144
Epoch 2/10
2520/2520 [==============================] - 5s 2ms/step - loss: 0.1241 - accuracy: 0.9659
Epoch 3/10
2520/2520 [==============================] - 5s 2ms/step - loss: 0.0753 - accuracy: 0.9769
Epoch 4/10
2520/2520 [==============================] - 5s 2ms/step - loss: 0.0569 - accuracy: 0.9790
Epoch 5/10
2520/2520 [==============================] - 5s 2ms/step - loss: 0.0524 - accuracy: 0.9796
Epoch 6/10
2520/2520 [==============================] - 5s 2ms/step - loss: 0.0525 - accuracy: 0.9778
Epoch 7/10
2520/2520 [==============================] - 5s 2ms/step - loss: 0.0772 - accuracy: 0.9776
Epoch 8/10
2520/2520 [==============================] - 5s 2ms/step - loss: 0.0497 - accuracy: 0.9781
Epoch 9/10
2520/2520 [==============================] - 6s 2ms/step - loss: 0.0519 - accuracy: 0.9794
Epoch 10/10
788/788 [==============================] - 1s 2ms/step - loss: 0.0459 

In [60]:
svc_c = svm.SVC()
svc_c.fit(X, y)

SVC()

In [61]:
sgd_c = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
sgd_c.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


SGDClassifier(max_iter=5)

## **Train Predictions**

In [62]:
nn_preds_train = model.predict(X)

788/788 [==============================] - 1s 1ms/step


In [63]:
svc_preds_train = svc_c.predict(X)

In [64]:
sgd_preds_train = sgd_c.predict(X)

## **Ensemble model**

In [69]:
pred_dicts = {'neural_network': list(nn_preds_train), 
              'svc': list(svc_preds_train),
              'sgd': list(sgd_preds_train)}
pred_dicts = pd.DataFrame.from_dict(pred_dicts)

In [70]:
ens_model = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
ens_model.fit(pred_dicts, y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


SGDClassifier(max_iter=5)

In [77]:
ens_train_pred = ens_model.predict(pred_dicts)

In [83]:
acc = sum(1 - abs(y - np.array(ens_train_pred))) / len(ens_train_pred)
acc

0.9858685296919657

## ***Test Prediction***

In [71]:
nn_preds = model.predict(test_data)

705/705 [==============================] - 1s 1ms/step


In [72]:
svc_preds = svc_c.predict(test_data)

In [73]:
sgd_preds = sgd_c.predict(test_data)

In [74]:
pred_dicts_test = {'neural_network': list(nn_preds), 
              'svc': list(svc_preds),
              'sgd': list(sgd_preds)}
pred_dicts_test = pd.DataFrame.from_dict(pred_dicts_test)

In [75]:
ens_model.predict(pred_dicts_test)

array([0, 0, 1, ..., 0, 1, 0])